### Rinex_V3_Parser_NAV

##### Author: Ajay Ragh
##### Data Source: GNSS Nav files
##### Data Format: RINEX Version 3 Nav files
##### Package Used: georinex (https://github.com/geospace-code/georinex)

##### Installing Packages

In [ ]:
!pip install georinex

##### importing packages

In [1]:
import georinex as gr
import numpy as np
import pandas as pd

#### Loading data from the rinex
1. Remove the (meas=band) parameter to load the entire rinex file. 
2. This parameter limits to load only the values from these variables (D1C and D5Q) for faster loading, useful for large rinex obs files.
3. The svid list is made to save the satellite vehicle id of all GPS satellites. this can be used in further code to only select data from GPS satellites.


In [2]:
file = "Data/input/sdr_data/sdr_20220419065643.nav"
navs = gr.load(file) 
hdr = gr.rinexheader(file)
sv = navs['sv']
sv = np.asarray(sv)
svid = []
for i in sv:
    char = str(i)
    if (char[0] == 'G'):
        svid.append(char)


##### Display the loaded data

In [3]:
navs

<xarray.Dataset>
Dimensions:           (sv: 10, time: 2)
Coordinates:
  * sv                (sv) <U3 'G02' 'G06' 'G11' 'G13' ... 'G19' 'G28' 'G30'
  * time              (time) datetime64[ns] 2022-04-19T07:59:44 2022-04-19T08...
Data variables: (12/28)
    SVclockBias       (time, sv) float64 nan nan nan ... 0.0001463 0.0003698 nan
    SVclockDrift      (time, sv) float64 nan nan nan ... -4.206e-12 nan
    SVclockDriftRate  (time, sv) float64 nan nan nan nan nan ... 0.0 0.0 0.0 nan
    IODE              (time, sv) float64 nan nan nan nan ... 16.0 42.0 17.0 nan
    Crs               (time, sv) float64 nan nan nan nan ... -49.25 132.1 nan
    DeltaN            (time, sv) float64 nan nan nan ... 4.166e-09 4.073e-09 nan
    ...                ...
    L2Pflag           (time, sv) float64 nan nan nan nan nan ... 0.0 0.0 0.0 nan
    SVacc             (time, sv) float64 nan nan nan nan nan ... 2.4 2.4 2.4 nan
    health            (time, sv) float64 nan nan nan nan ... 0.0 0.0 63.0 nan
    TGD               (time, sv) float64 nan nan nan ... -1.118e-08 nan
    IODC              (time, sv) float64 nan nan nan nan ... 16.0 42.0 17.0 nan
    TransTime         (time, sv) float64 nan nan nan ... 1.979e+05 1.979e+05 nan
Attributes:
    version:    3.02
    svtype:     ['G']
    rinextype:  nav
    filename:   sdr_20220419065643.nav

##### Display the RInex Header

In [4]:
hdr

{'version': 3.02,
 'filetype': 'N',
 'rinextype': 'nav',
 'systems': 'M',
 'PGM / RUN BY / DATE': 'GNSS-SDRLIB v1.1                        20220419 065643 UTC '}

In [5]:
def extract_time_info(svid,array):
    toe = []
    gpsweek = []
    transtime = []
    time = []
    data = array.sel(sv=svid)
    toe_temp = data['Toe']
    toe_temp = np.asarray(toe_temp)
    gpsweek_temp = data['GPSWeek']
    gpsweek_temp = np.asarray(gpsweek_temp)
    transtime_temp = data['TransTime']
    transtime_temp = np.asarray(transtime_temp)
    time_temp = data['time']
    time_temp = np.asarray(time_temp)
    for i in range(time_temp.shape[0]):
        if(str(toe_temp[i]) != 'nan'):
            toe.append(toe_temp[i])
            time.append((time_temp[i]))
        if(str(gpsweek_temp[i]) != 'nan'):
            gpsweek.append(gpsweek_temp[i])
        if(str(transtime_temp[i]) != 'nan'):
            transtime.append(transtime_temp[i])
#     doppler = np.asarray(doppler)
    return toe,gpsweek,transtime,time

In [11]:
for i in svid:
    toe,gpsweek,transtime,time = extract_time_info(i,navs)
    datetime = time[0]
    year = str(datetime)[0:4]
    month = str(datetime)[5:7]
    day = str(datetime)[8:10]
    time_data = pd.DataFrame(list(zip(time,toe,gpsweek,transtime)),columns=['Epoch','Time of Ephmeris','GPSWeek','Transmission Time'])
    time_data.to_csv('Data/output/sdr/nav/time_info_{}_{}{}{}.txt'.format(i,year,month,day),header=None, index=None, sep=' ')